### Test collection locally

In [1]:
from typing import List, Union

import pystac
from stactools.ecmwf_forecast import stac

from pctasks.core.models.task import WaitTaskResult
from pctasks.core.storage import StorageFactory
from pctasks.dataset.collection import Collection

class EcmwfCollection(Collection):
    @classmethod
    def create_item(
        cls, asset_uri: str, storage_factory: StorageFactory
    ) -> Union[List[pystac.Item], WaitTaskResult]:
        asset_storage, asset_path = storage_factory.get_storage_for_file(asset_uri)

        grib2_href = asset_storage.get_url(asset_path)
        index_href = grib2_href.rsplit(".", 1)[0] + ".index"
        item = stac.create_item([grib2_href, index_href], split_by_step=True)

        return [item]

/anaconda/envs/py38_default/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


In [2]:
url = 'blob://ai4edataeuwest/ecmwf/20230929/00z/0p4-beta/wave/20230929000000-0h-wave-fc.grib2'
[item] = EcmwfCollection.create_item(url, StorageFactory())

/anaconda/envs/py38_default/lib/python3.8/site-packages/kerchunk/combine.py:269: UserWarning: Concatenated coordinate 'valid_time' contains less than expectednumber of values across the datasets: [1695945600]
  warnings.warn(


In [3]:
%%time
import fsspec
import xarray as xr
m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'])
ds = xr.open_dataset(m, engine='zarr')
ds['swh'].mean()

CPU times: user 55.2 ms, sys: 8.76 ms, total: 64 ms
Wall time: 153 ms


<timed exec>:4: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.


<xarray.DataArray 'swh' ()>
array(2.55190232)